In [ ]:
# Make json to csv

import json
import pandas as pd
from glob import glob

file_list = glob("./*.json")

for file_path in file_list : 
    # Load the JSON file
    with open(file_path, "r") as file:
        data = json.load(file)

    # Define the categories (16 labels)
    categories = [
        "subject_consistency", "background_consistency", "temporal_flickering", "motion_smoothness",
        "dynamic_degree", "aesthetic_quality", "imaging_quality", "object_class", "multiple_objects",
        "human_action", "color", "spatial_relationship", "scene", "temporal_style",
        "appearance_style", "overall_consistency"
    ]

    # Dictionary to store video scores
    video_scores = {}

    # Extract data
    for category in categories:
        if category in data and isinstance(data[category], list) and len(data[category]) > 1:
            for entry in data[category][1]:  # The second element contains the video details
                video_path = entry["video_path"].split("/")[-1]  # Extract the filename
                score = entry.get("video_results", 0)  # Get score or default to 0
                
                if video_path not in video_scores:
                    video_scores[video_path] = {cat: 0 for cat in categories}  # Initialize with 0
                
                video_scores[video_path][category] = score

    # Convert to DataFrame
    df = pd.DataFrame.from_dict(video_scores, orient="index").reset_index()
    df.rename(columns={"index": "video_path"}, inplace=True)
    df.replace({True: 1, False: 0}, inplace=True)

    # Save the DataFrame to a CSV file
    csv_file_path = file_path.replace(".json", ".csv")
    df.to_csv(csv_file_path, index=False)

    # Provide the file path for download
    print(csv_file_path)


./Merged_Video.csv
./Result_merged_5videos.csv


/tmp/ipykernel_34842/585429858.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({True: 1, False: 0}, inplace=True)
/tmp/ipykernel_34842/585429858.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({True: 1, False: 0}, inplace=True)


In [ ]:
# Analyze Model Weight

import torch
import numpy as np

# 1️⃣ 저장된 모델 가중치 로드
pth_file = "models_scale/final_model_V5.pth"  # 저장된 .pth 파일 경로
model_weights = torch.load(pth_file)

# 2️⃣ 특정 Layer의 가중치 출력 (예: 첫 번째 Linear Layer)
layer_name = "fc1.weight"  # 확인하고 싶은 레이어 이름
if layer_name in model_weights:
    weight_tensor = model_weights[layer_name]  # PyTorch Tensor
    weight_numpy = weight_tensor.cpu().numpy()  # NumPy 배열로 변환
    print(weight_numpy)  # 출력
else:
    print(f"Layer '{layer_name}' not found in model weights.")


[[-0.09633601 -0.07910314 -0.02594274 ...  0.10128367 -0.2205689
   0.00884959]
 [-0.16278568 -0.09682745 -0.11542109 ... -0.1915836   0.14608547
   0.07320482]
 [-0.04110768 -0.15510577  0.04451498 ...  0.19898316 -0.02296776
  -0.18367738]
 ...
 [ 0.10643771  0.03972483  0.1100437  ... -0.13935032 -0.02709863
  -0.09881777]
 [-0.1876798   0.18063924 -0.15056106 ...  0.15770549  0.17539868
  -0.08890405]
 [ 0.151283    0.13848668  0.15440562 ...  0.06647527  0.00652781
  -0.13567278]]


/tmp/ipykernel_6395/454693564.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights = torch.load(pth_file)


In [ ]:
# Analyze Model Structure

import torch

class AttentionBlock(torch.nn.Module):
    def __init__(self, input_dim):
        super(AttentionBlock, self).__init__()
        self.query = torch.nn.Linear(input_dim, input_dim)
        self.key = torch.nn.Linear(input_dim, input_dim)
        self.value = torch.nn.Linear(input_dim, input_dim)
        self.softmax = torch.nn.Softmax(dim=-1)
    
    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        # Scaled Dot-Product Attention
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (x.shape[-1] ** 0.5)
        attn_weights = self.softmax(attn_scores)
        out = torch.matmul(attn_weights, V)

        return out


class AttentionModel(torch.nn.Module):
    def __init__(self, input_dim=16, hidden_dim=64):
        super(AttentionModel, self).__init__()
        self.attention = AttentionBlock(input_dim)
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        x = self.attention(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 가중치 확인
model = AttentionModel()
print(model)  # attention 블록의 가중치 없음

    

AttentionModel(
  (attention): AttentionBlock(
    (query): Linear(in_features=16, out_features=16, bias=True)
    (key): Linear(in_features=16, out_features=16, bias=True)
    (value): Linear(in_features=16, out_features=16, bias=True)
    (softmax): Softmax(dim=-1)
  )
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [ ]:
# Replace Frame with Random Noise

import cv2
import numpy as np
import random
import os

# === 설정: 동영상이 있는 폴더 목록 ===
ratio = 0.2
folders = ["./mov_select/Frozen", "./mov_select/Frozen2", "./mov_select/Up", "./mov_select/Inside_Out", "./mov_select/Zootopia"]
output_folder = f"random_noisy_videos_{ratio}"

# 결과 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

def generate_noise_frame(width, height):
    """랜덤한 노이즈 프레임 생성"""
    return np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)

def process_video_random_noise_frames(input_video, output_video, ratio):
    """랜덤한 절반의 프레임을 노이즈로 대체하는 동영상 생성"""
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print(f"❌ 동영상을 열 수 없습니다: {input_video}")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # 랜덤하게 절반의 프레임을 노이즈로 대체
    noise_frames_idx = set(random.sample(range(frame_count), int(frame_count * ratio)))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx in noise_frames_idx:
            frame = generate_noise_frame(width, height)
        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"✅ 랜덤 프레임 노이즈 추가 완료: {output_video}")



# === 모든 폴더의 동영상 처리 ===
for idx in range(1,3) : 
    for folder in folders:
        files = 0
        for file in os.listdir(folder):
            if file.endswith(('.mp4', '.mov', '.avi')):
                input_path = os.path.join(folder, file)
                output_folder = f"random_noisy_videos_{ratio}_{idx}"
                output_path = os.path.join(output_folder, f"random_noisy_{file}")
                process_video_random_noise_frames(input_path, output_path, ratio*idx)
                files = files + 1
                if files > 100 : break

print("🎉 모든 동영상에 랜덤 프레임 노이즈 추가 완료!")


In [ ]:
# Remove Files Larger than 70MB (Ubuntu Command)

!find ./random_noisy_videos_0.6 -type f -size +70M -exec rm -v {} +

In [ ]:
# Split Video

import os
import glob
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from scenedetect.video_splitter import split_video_ffmpeg

# 📂 입력 폴더 (전체 영상 탐색)
threshold = 50
video_folder = "mov_source/"
output_base_folder = f"mov_split_{threshold}/"

# 🎥 지원하는 비디오 확장자 목록
video_extensions = ["*.mp4", "*.mkv", "*.avi", "*.mov"]

# 🎯 입력 폴더에서 모든 비디오 파일 찾기
video_files = []
for ext in video_extensions:
    video_files.extend(glob.glob(os.path.join(video_folder, ext)))

if not video_files:
    print("⚠️ 처리할 영상이 없습니다.")
    exit()

print(f"총 {len(video_files)}개의 영상을 처리합니다.")

# 🎬 모든 비디오 파일을 순차적으로 처리
for video_path in video_files:
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_folder = os.path.join(output_base_folder, video_name)

    # 📂 출력 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    print(f"\n🎥 '{video_name}' 영상 처리 중...")

    # 🎬 VideoManager 및 SceneManager 설정
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()

    # 🟡 ContentDetector 사용 (기본 감도: threshold=30)
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    # 🎥 영상 분석 시작
    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager, show_progress=True)

    # 🎯 탐지된 장면 리스트 가져오기
    scene_list = scene_manager.get_scene_list()
    csv_file_path = os.path.join(output_folder, f"{video_name}-Scenes.csv")
    with open(csv_file_path, "w") as f:
        f.write("Scene Number,Start Time (frames),End Time (frames)\n")
        for i, scene in enumerate(scene_list):
            start, end = scene
            f.write(f"{i+1},{start.get_frames()},{end.get_frames()}\n")

    # print(scene_list)
    print(f"총 {len(scene_list)}개의 장면이 탐지되었습니다.")

    # 💾 FFmpeg을 사용해 각 장면별로 영상 파일로 저장
    split_video_ffmpeg(
        input_video_path=video_path,
        scene_list=scene_list,
        output_dir=output_folder,
        output_file_template=f"{video_name}-Scene-$SCENE_NUMBER.mp4",
        show_progress=True
    )

    print(f"✅ '{video_name}'의 장면별 영상이 {output_folder} 폴더에 저장되었습니다.")

print("\n🎉 모든 영상 처리가 완료되었습니다!")


In [ ]:
# Video Grid Show

import cv2
import numpy as np
import os

# 동영상이 있는 폴더 경로
video_folder = "./Supplement"  # 동영상 폴더 경로
output_image_path = "./output_collage3.jpg"  # 최종 출력 이미지 경로

# 동영상 파일 리스트 (0.mp4 ~ 9.mp4)
video_files = [os.path.join(video_folder, f"{i}.mp4") for i in range(10)]

# 설정값
num_frames = 6  # 한 동영상에서 추출할 프레임 수
frame_size_scale = 20
frame_width = 16 * frame_size_scale  # 각 프레임의 가로 크기 (조절 가능)
frame_height = 19 * frame_size_scale   # 각 프레임의 세로 크기 (조절 가능)
gap = 10  # 줄 간 공백 (픽셀)

# 각 동영상에서 8개의 프레임을 캡처하여 한 줄로 합친 결과 저장
rows = []

for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps  # 전체 동영상 길이 (초)

    if frame_count < num_frames:
        print(f"⚠️ {video_file}는 {num_frames} 프레임 이상이 필요합니다. (총 {frame_count} 프레임)")
        continue

    # 1/8씩 시간 간격을 나누어 프레임 선택
    time_intervals = [(duration * i / num_frames) for i in range(num_frames)]
    frame_indices = [int(t * fps) for t in time_intervals]  # 초 → 프레임 번호 변환

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (frame_width, frame_height))  # 크기 조정
            frames.append(frame)
    
    cap.release()

    if len(frames) == num_frames:
        row_image = np.hstack(frames)  # 한 줄로 연결
        rows.append(row_image)

# 모든 행을 세로로 결합하면서 줄 간 간격 추가
if rows:
    blank_space = np.ones((gap, row_image.shape[1], 3), dtype=np.uint8) * 255  # 공백(흰색)
    final_image = rows[0]

    for row in rows[1:]:
        final_image = np.vstack((final_image, blank_space, row))

    # 결과 저장 및 출력
    cv2.imwrite(output_image_path, final_image)
    print(f"✅ 최종 이미지가 저장되었습니다: {output_image_path}")
else:
    print("⚠️ 이미지 생성에 실패했습니다. 동영상 파일을 확인하세요.")
